# Uncertainty Estimation in Large Language Models to Support Biodiversity Conservation

**María Mora-Cross and Saúl Calderón-Ramírez**
Costa Rica Institute of Technology
{maria.mora, sacalderon}@itcr.ac.cr

July - December, 2023

### Abstract

Large Language Models (LLM) provide significant value in question answering (QA) scenarios and have practical application in complex decision-making contexts, such as biodiversity conservation. However, despite substantial performance improvements, they may still produce inaccurate outcomes. Consequently, incorporating uncertainty quantification alongside predictions is essential for mitigating the potential risks associated with their use. This study introduces an exploratory analysis of the application of Monte Carlo Dropout (MCD) and Expected Calibration Error (ECE) to assess the uncertainty of generative language models. To that end, we analyzed two publicly available language models (Falcon-7B and DistilGPT- 2). Our findings suggest the viability of employing ECE as a metric to estimate uncertainty in generative LLM. 

The findings from this research contribute to a broader project aiming at facilitating free and open access to standardized and integrated data and services about Costa Rica’s biodiversity to support the development of science, education, and biodiversity conservation.

**Detailed description of the experiment is available at:**
Maria Mora-Cross and Saul Calderon-Ramirez (2024). Uncertainty Estimation in Large Language Models to Support Biodiversity Conservation. Proceedings of the 2024 Conference of the North American Chapter of the Association for
Computational Linguistics: Human Language Technologies.  

This notebook was used to run the experiments detailed in the associated publication.

Please note that after the project's completion, the authors discovered that the ELI5 dataset is no longer available on Hugging Face. The dataset page has the message: "Dataset eli5 is defunct and no longer accessible due to unavailability of the source data".

In [7]:
# Libraries
import torch
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import random
import torch.nn.functional as F
from datetime import date

# sklearn
from sklearn.preprocessing import MinMaxScaler

# imbalace library
from imblearn.under_sampling import RandomUnderSampler

# Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config
from huggingface_hub import notebook_login
import datasets
from datasets import load_dataset
from datasets import DatasetDict
from collections import defaultdict

# Metrics
from datasets import load_metric
from transformers import pipeline

import matplotlib.pyplot as plt

# hugging face metrics
from evaluate import load

# It is recomended to install accelerate 0.21
import accelerate
accelerate.__version__


# Utilities 
from utilityV02 import *
setup()


Using transformers v4.36.2
Using datasets v2.14.4


In [8]:
# Variables
device = "cuda" if torch.cuda.is_available() else "cpu"


In [17]:
# Accessing huggingface

notebook_login()

### Load and preprocess the dataset

**Dataset:**

The **ELI5 dataset is an English-language dataset of questions and answers gathered from three subreddits (service to manage communities)** where users ask factual questions requiring paragraph-length or **longer answers**. The dataset was created to support the task of **open-domain long form abstractive question answering, and covers questions about:** general topics in its r/explainlikeimfive subset, **science in it r/askscience subset**, and History in its r/AskHistorians subset.

**Data Fields**

- q_id: a string question identifier for each example, corresponding to its ID in the Pushshift.io Reddit submission dumps.
- subreddit: One of explainlikeimfive, askscience, or AskHistorians, indicating which subreddit the question came from
- title: title of the question, with URLs extracted and replaced by URL_n tokens
- title_urls: list of the extracted URLs, the nth element of the list was replaced by URL_n
- selftext: either an empty string or an elaboration of the question
- selftext_urls: similar to title_urls but for self_text
- answers: **a list of answers**, each answer has:
   - a_id: a string answer identifier for each answer, corresponding to its ID in the Pushshift.io Reddit comments dumps.
   - text: the answer text with the URLs normalized
   - score: the number of upvotes the answer had received when the dumps were created
- answers_urls: a list of the extracted URLs. All answers use the same list, the numbering of the normalization token continues across answer texts

Fan, A. Jernite, Y., Perez, E., Grangier, D., Weston, J & Aulihttps, M. ().
Dataset Card for ELI5.  https://huggingface.co/datasets/eli5

In [7]:
# Exploring the contents of the dataset
eli5_test = load_dataset("eli5", split="test_asks[:]" )
print(eli5_test)

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 4462
})


In [9]:
print(eli5_test[0]['answers']['text'])
print(eli5_test[0]['answers']['a_id'])

['Muscles are grouped in either voluntary or involuntary muscle. From there it breaks down into skeletal\n muscle ( biceps triceps etc) which is considered voluntary, smooth muscle is involuntary, this includes the muscles in your intestines to digest food or pupils in your eyes to dilatue or not, and lastly to answer your question there is cardiac muscle. It is also involuntary. It is able to generate its own stimulus to contract when i needed. \n\n\nSorry for errors. On phone :p', "Cardiac tissue isn't under voluntary control. The parts of the brain that control things like breathing, digestion, heartbeat ect are controllable only to a certain extent. For example, if you concentrate you can slow your heart rate, but it will keep beating. Similarly you can control your breathing, but you can't make yourself suffocate by holding your breath, because eventually your body will force you to breath again. I'm not sure how this has evolved, but it certainly works out for the best that way. 

In [11]:
# How the data looks like

sample_seq = 4461

#Question
print(eli5_test[sample_seq]['title'])

print("==============================")
print("Cantidad de respuestas", len(eli5_test[sample_seq]['answers']['text']))
print("==============================")

# answer 10 for a question
print(eli5_test[sample_seq]['answers']['text'][1])
print(eli5_test[sample_seq]['answers']['score'])

Are there any limits to what science can explain?
Cantidad de respuestas 4
Science attempts to observe and explain the facts of the universe. However, I would argue (though some may disagree) that it is beyond the realm of science to answer questions of value. In other words, science seeks to understand the origin of the universe but is not interested in providing any kind of philosophical rationale. Nor is science interested in/capable of determining whether or not a work of art is good or if your life has any meaning. Even though we may someday fully understand the neurochemical mechanisms and evolutionary prompts for the human creation of value, it is up to us to assign any significance to it. That's why people who say that either science or spirituality/philosophy must eliminate the other are naïve -- the two should stick to asking entirely different sets of questions.
[8, 7, 5, 3]


### Load the model, the tokenizer and define the model dropout


In [ ]:
# 1)For GPT-2
# Load the model, the tokenizer and define the model dropout
model_name_tokenizer = "distilgpt2"
model_name = "mariamoracrossitcr/distilgpt2_finetuneWithEli5V2"

#config = GPT2Config.from_pretrained(model_name_tokenizer)
#model = AutoModelForCausalLM.from_pretrained(model_name, config=config).to(device)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Load a DistilGPT2 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_tokenizer)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [12]:
# 2) For Falcon-2b
# Falcon 7B was quantized using 4 bits and saved locally

from transformers import  FalconForCausalLM, BitsAndBytesConfig
#model = "tiiuae/falcon-7b"
model_name = "./FineTunningFalcon/results06"
model_name_tokenizer = "./FineTunningFalcon/results06"

tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    #trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Calibration

Process:
- Call to the calibration function that executes the algorithm and returns the three elements required to perform the ECE:
    - model_confidence_per_sample,
    - avg_predictions_per_sample,
    - variance_predictions_per_sample
- The results are normalized with MinMaxScaler.
- Data is saved on the ECE database.

In [13]:
### Calibration for an experiment
# Hyperparameters
model_type = 'falcon'

#### with dropout
dropout = 0.04
#####
answer_length = 512

#falcon 7b
# 5.2. Generating Text with Contrastive Search: : https://huggingface.co/blog/introducing-csearch
penalty_alpha = 0.1 
topk=5

#bertscore 
bertscore_lang = 'en'

# For gpt2
topp=0.95

# Random selection of samples 
samples_test = select_samples(eli5_test, 500)

num_bins =10

num_repetions_per_sample = 10

experiment_id = 22

# CALIBRATION
model_confidence_per_sample, avg_predictions_per_sample, variance_predictions_per_sample  = \
            calibration (num_bins, dropout, samples_test, num_repetions_per_sample, tokenizer, model, \
                         answer_length,topk, topp,  experiment_id, model_type, penalty_alpha,\
                         bertscore_lang, padToken_id=50256)


# Update database
""" Postprocesing and statistics
    For an experiment_id update database ECE
       Read data form QUESTION and SAMPLING tables, normalize perplexity and divide it into ranges.
       Save data in QUESTION_SAMPLING
"""
# 11 bins generate 10 clases using linespace
num_bins = 11
postprocessing_tasks_QUESTION_SAMPLING(experiment_id, num_bins)


# Extract data from QUESTION_SAMPLING from the database (nromalized data) and save data in
# Question_sampling_stat table
query_text = "select q.q_id, q.experiment_id,  q.mod_perplexity_norm, \
                               q.samp_perplexity_norm, q.bin from question_sampling q \
                                  where q.experiment_id = " + str(experiment_id)
postprocessing_tasks_QUESTION_SAMPLING_STAT(query_text)

/home/mmora/Applications/anaconda3/envs/ml_env/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


+-----+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
| q_id|experiment_id|experiment_date|            question|        model_answer| perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+-----+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
|k9e7s|           22|     2024-01-10|I had to ask in /...|[I had to ask in ...|3.908203125|   5|0.95|   0.04|        512|0.8033592104911804|0.7652708292007446|0.8454376459121704|
+-----+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+

+-----+-------+-----+--------------------+-------------+
| q_id|   a_id|score|         true_answer|exper

+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
| q_id|experiment_id|experiment_date|            question|        model_answer|perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|j0ebk|           22|     2024-01-10|Is it plausible t...|[Is it plausible ...|2.83203125|   5|0.95|   0.04|        512|0.7868825197219849|0.7403596043586731|0.8396442532539368|
+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+

+-----+-------+-----+--------------------+-------------+
| q_id|   a_id|score|         true_answer|experiment

+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
|  q_id|experiment_id|experiment_date|            question|        model_answer| perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
|1tj6fo|           22|     2024-01-10|Since neutrinos a...|[Since neutrinos ...|3.017578125|   5|0.95|   0.04|        512|0.8281917572021484|0.8377081751823425|0.8188890218734741|
+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+

+------+-------+-----+--------------------+-------------+
|  q_id|   a_id|score|         true_answe

+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
| q_id|experiment_id|experiment_date|            question|        model_answer|perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|ixy3v|           22|     2024-01-10|Need Help Making ...|[Need Help Making...|5.16015625|   5|0.95|   0.04|        512|0.8182324767112732|0.8197071552276611|0.8167629837989807|
+-----+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+

+-----+-------+-----+--------------------+-------------+
| q_id|   a_id|score|         true_answer|experiment

...

+------+-------+-----+--------------------+-------------+
|  q_id|   a_id|score|         true_answer|experiment_id|
+------+-------+-----+--------------------+-------------+
|5jkwtx|dbgwyiy|  455|If you're wonderi...|           22|
|5jkwtx|dbh611y|  323|How does someone ...|           22|
|5jkwtx|dbh5ut4|  188|Is it possible th...|           22|
|5jkwtx|dbh0x9p|   90|What effect, if a...|           22|
|5jkwtx|dbhbpok|   76|I read the BBC ar...|           22|
|5jkwtx|dbh7oj8|   62|Why is there such...|           22|
|5jkwtx|dbgy1pw|   40|Would a black hol...|           22|
|5jkwtx|dbh7rvp|   30|The paper says th...|           22|
|5jkwtx|dbhv6dg|   23|What are the chan...|           22|
|5jkwtx|dbhkyyt|   21|Hey. This isn't e...|           22|
|5jkwtx|dbh8kay|   20|Observation here,...|           22|
|5jkwtx|dbhc0zu|   15|Correct me if I'm...|           22|
|5jkwtx|dbh53sn|   15|Here's yet anothe...|           22|
|5jkwtx|dbh9ndf|   11|any ideas why mat...|           22|
|5jkwtx|dbhcsk

+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|  q_id|experiment_id|experiment_date|            question|        model_answer|perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|287l02|           22|     2024-01-18|why would natural...|[why would natura...|6.87890625|   5|0.95|   0.04|        512|0.8206769227981567|0.8242698311805725|0.8171151876449585|
+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+

+------+-------+-----+--------------------+-------------+
|  q_id|   a_id|score|         true_answer|exp

+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
|  q_id|experiment_id|experiment_date|            question|        model_answer| perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+
|1ggjcg|           22|     2024-01-18|Do "eye crusties"...|[Do "eye crusties...|3.986328125|   5|0.95|   0.04|        512|0.8182999491691589|0.7983654737472534|0.8392554521560669|
+------+-------------+---------------+--------------------+--------------------+-----------+----+----+-------+-----------+------------------+------------------+------------------+

+------+-------+-----+--------------------+-------------+
|  q_id|   a_id|score|         true_answe

+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|  q_id|experiment_id|experiment_date|            question|        model_answer|perplexity|topk|topp|dropout|text_length|                f1|         precision|            recall|
+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+
|1necyi|           22|     2024-01-18|Can people teach ...|[Can people teach...|2.78515625|   5|0.95|   0.04|        512|0.8291912078857422|0.8168652057647705|0.8418948650360107|
+------+-------------+---------------+--------------------+--------------------+----------+----+----+-------+-----------+------------------+------------------+------------------+

+------+-------+-----+--------------------+-------------+
|  q_id|   a_id|score|         true_answer|exp

+------+-------------+--------------+-----------+---------------+-------------------+--------------------+---+
|  q_id|experiment_id|mod_perplexity|sampling_id|samp_perplexity|mod_perplexity_norm|samp_perplexity_norm|bin|
+------+-------------+--------------+-----------+---------------+-------------------+--------------------+---+
|100ofr|           22|        5.6875|          6|     4.52734375|  0.368220742150333| 0.08115789473684212|  4|
|100ofr|           22|        5.6875|          7|     4.66015625|  0.368220742150333| 0.08473684210526317|  4|
|100ofr|           22|        5.6875|          8|     3.40234375|  0.368220742150333|0.050842105263157904|  4|
|100ofr|           22|        5.6875|          9|     3.43359375|  0.368220742150333|  0.0516842105263158|  4|
|100ofr|           22|        5.6875|         10|     4.25390625|  0.368220742150333| 0.07378947368421054|  4|
|100ofr|           22|        5.6875|          1|     4.06640625|  0.368220742150333| 0.06873684210526315|  4|
|

## References
Fan, A., Jernite, Y., Perez, E., Grangier, D., Weston, J. & Auli, M. (2019). ELI5: Long Form Question Answering. Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics, pages 3558–3567. https://aclanthology.org/P19-1346.pdf


Zhang, T., Kishore, V., Wu, F., Weinberger, K., & Artzi, Y. (2020). Bertscore: Eval- uating text generation with bert. https://openreview.net/pdf?id=SkeHuCVFDr

Michael, H. & Ondrej, B.(2021). Fine-Grained Analysis of BERTScore. Proceedings of the Sixth Conference on Machine Translation (WMT), pages 507–517. https://aclanthology.org/2021.wmt-1.59.pdf
